In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import io
from PIL import Image
import hashlib

# download wikipage
#wikipage = "https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependent_territories_by_continent_(data_file)"
#result = requests.get(wikipage)

# if successful parse the download into a BeautifulSoup object, which allows easy manipulation 
#if result.status_code == 200:
#    soup = BeautifulSoup(result.content, "html.parser")
    
# find the object with HTML class wikitable sortable
#table = soup.find('table',{'class':'wikitable sortable'})

# loop through all the rows and pull the text
#new_table = []
#for row in table.find_all('tr')[1:]:
#    column_marker = 0
#    columns = row.find_all('td')
#    new_table.append([column.get_text() for column in columns])
    
#df = pd.DataFrame(new_table, columns=['ContinentCode','Alpha2','Alpha3','PhoneCode','Name'])
#df['Name'] = df['Name'].str.replace('\n','')
#df

In [2]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = 'C:/Users/alexa/Documents/GitHub/anic0077.github.io/misc/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
wd.quit()
#wd.get('https://google.com')
#search_box = wd.find_element_by_css_selector('input.gLFyf')
#search_box.send_keys('Dogs')

In [3]:
#wd.quit()

In [4]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:float=0.5):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(1)
            #return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [5]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [6]:
#combining above 2 functions
def search_and_download(search_term:str,driver_path:str,number_images:int,target_path='./images'):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [35]:
search_term = 'lavender'

search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH,
    number_images = 500
)

Found: 100 search results. Extracting links from 0:100
Found: 161 image links, looking for more ...
Found: 212 search results. Extracting links from 100:212
Found: 338 image links, looking for more ...
Found: 312 search results. Extracting links from 212:312
Found: 492 image links, looking for more ...
Found: 412 search results. Extracting links from 312:412
Found: 501 image links, done!
ERROR - Could not save https://1v1d1e1lmiki1lgcvx32p49h8fe-wpengine.netdna-ssl.com/wp-content/uploads/2019/09/1568352085-inifiniti-car-getty-960x540.jpg - cannot identify image file <_io.BytesIO object at 0x000002D120196130>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRZ6FPRLnSKFPTmkbJ5XLg-kxf5XSk-JEc4-A&usqp=CAU - as ./images\car\7a31359504.jpg
SUCCESS - saved https://www.kia.com/content/dam/kwcms/au/en/images/category/7-seater-family-suv-kia-sorento.jpg - as ./images\car\df6bd6a9c7.jpg
SUCCESS - saved https://static01.nyt.com/images/2021/03/26/business/25wheels/00CARS-COMBO-